In [15]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action="ignore")


In [16]:
import requests
from bs4 import BeautifulSoup
import time # sleep() 메소드를 사용하기 위해 import한다.

In [35]:
# 1건의 대화 내용을 저장하는 클래스 -> 질문과 답변이 한 쌍으로 저장되는 클래스
class Conversation:
    def __init__(self, question, answer):
        self.question = question
        self.answer = answer
    def __str__(self):
        return (f"Q : {self.question}\nA : {self.answer}")

In [36]:
c = Conversation("어서와 방탄은 처음이지???", "응으으으 ;;;")
print(c)

Q : 어서와 방탄은 처음이지???
A : 응으으으 ;;;


In [37]:
def getSubject() :
    targetSite = "https://basicenglishspeaking.com/daily-english-conversation-topics/"
    request = requests.get(targetSite)
    html = request.text
    soup = BeautifulSoup(html, "html.parser")
    
    subjects = []
    contentLinks = []
    divs = soup.findAll("div", {"class": "thrv_text_element"})
    for div in divs :
        a = div.findAll("a")
        for text in a:
            subjects.append(text.text)
            contentLinks.append(text.get("href"))
        #--------- for content
    #-------- for div
    return subjects, contentLinks
    

In [38]:
subjects, contentLinks = getSubject()

In [43]:
#대화주제에 따른 전체 대화 내용(Conversation 클래스 객체)을 저장할 빈 리스트를 선언한다.
conversation = []

#대화 주제의 갯수만큼 반복하며 각각의 대화 주제에 따른 대화 내용을 읽어서 저장한다.
for i in range(len(subjects[:])):
    conversation.append("{:2d}, {}: {}".format( (i + 1), subjects[i], contentLinks[i] ))
    #대화 주제별 대화 내용을 크롤링할 페이지를 요청한다.
    request = requests.get(contentLinks[i])
    
    #time 모듈을 import하면 sleep() 메소드를 사용할 수 있다.
    #sleep() 메소드는 인수로 지정한 시간만큼 프로그램을 멈춘다. -> 시간은 초 단위로 지정한다.
    #대형 포털 사이트는 짧은 간격으로 많은 요청이 들어오면 자기네 사이트가 공격당하는 것으로 판단할 수 있고,
    #자동화된 크롤링을 할 경우 페이지가 로딩되는 시간만큼 기다리기 위해 일정한 간격을 두고 정보를 요청한다.
    time.sleep(1)
    
    html = request.text
    soup = BeautifulSoup(html, "html.parser")
    
    #대화 주제에 따른 대화 내용은 플레이 버튼이 포함된 class 속성의 속성값이 "compact_audio_player_wrapper"인
    #<div> 태그의 바로 다음 형제이다.
    buttons = soup.findAll( "div", {"class": "compact_audio_player_wrapper"} )
    #print(len(buttons)) # 플레이 버튼의 갯수 -> 크롤링할 문장의 갯수
    
    #크롤링할 문장의 갯수만큼 반복한다.
    for button in buttons:
        #index() 메소드는 전체 객체 중엣 인수로 지정한 객체의 index를 얻어온다.
        #buttons.index(button) -> 전체 플레이 버튼(buttons)에서 특정 플레이 버튼(button)의 index를 얻어온다.
        #index() 메소드의 실행 경과가 짝수면 질문이고, 홀수면 답변이다.
        #print(buttons.index(button))
        
        #크롤링할 문장은 플레이 버튼이 포함된 <div> 태그 내부에 있지 않고, 다음 형제로 작성되어있다.
        #next_sibling : 바로 다음 형제
        #previous_sibling : 바로 이전 형제
        if buttons.index(button) % 2 == 0: # index가 짝수인가? 홀수인가?
            #print(f"질문 -> {button.next_sibling}")
            question = button.next_sibling
        else :
            #print(f"답변 -> {button.next_sibling}")
            answer = button.next_sibling
            #답변이 있어야 대화 1건이 완성된다.
            #Conversation 클래스 객체를 만드는 작업은 질문과 답변이 한 쌍이 되는 순간 실행한다.
            #질문과 답변이 한 쌍이 되었으므로 Conversation 클래스 객체를 생성하고 질문과 답변을 저장한다.
            c = Conversation(question, answer)
            #질문과 답변이 한 쌍으로 저장된 Conversation 클래스 객체를 대화 내용을 기억할 
            #conversation 리스트에 추가한다.
            conversation.append(c)
        # --------- if
    #---------- for
    
        
        

In [44]:
for c in conversation:
    print(c)
    print("*" * 100)

 1, Family: https://basicenglishspeaking.com/family/
****************************************************************************************************
Q : How many people are there in your family?
A : There are 5 people in my family: my father, mother, brother, sister, and me.
****************************************************************************************************
Q : Does your family live in a house or an apartment?
A : We live in a house in the countryside.
****************************************************************************************************
Q : What does your father do?
A : My father is a doctor. He works at the local hospital.
****************************************************************************************************
Q : How old is your mother?
A : She is 40 years old, 1 year younger than my father.
****************************************************************************************************
Q : Do you have any siblings? What’s his/her 